<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/08_nlp_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-10-26 07:15:58--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-10-26 07:15:59 (106 MB/s) - ‘helper_functions.py.1’ saved [6733/6733]



In [4]:
from helper_functions import unzip_data,plot_loss_curves,compare_historys,create_tensorboard_callback

In [5]:
# Get the Dataset from kaggle NLP with Disaster Tweets
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-10-26 07:15:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 64.233.170.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K   703KB/s    in 0.8s    

2025-10-26 07:16:00 (703 KB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [6]:
unzip_data("nlp_getting_started.zip")

In [7]:
# Become One with Dataset (Visualize)
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [9]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [10]:
# Shuffle Dataset
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
# What does Test Dataset Look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
# How many examples of each classes
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [13]:
# How many Total Samples
len(train_df),len(test_df)

(7613, 3263)

# **Visualize some random training examples**

In [14]:
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f'Target : {target} \n ',"(real disaster)" if target>0 else "not real disaster")
    print(f'Text : \n {text}')
    print('---\n')

Target : 0 
  not real disaster
Text : 
 Fatality  https://t.co/407V1y4HHg
---

Target : 0 
  not real disaster
Text : 
 I JUST SCREAMED SIDJSJDJEKDJSKDJD . I CANT STAND YOU ?? https://t.co/0Vcsafx9bY
---

Target : 0 
  not real disaster
Text : 
 Court back in session. Testimony continues with med. examiner discussing gunshot wounds #KerrickTrial
---

Target : 1 
  (real disaster)
Text : 
 #NJTurnpike å_ #NJTurnpike Reopens Hours After Truck Fire In? http://t.co/oABJZtbVyZ http://t.co/GPBXRrDc07
---

Target : 1 
  (real disaster)
Text : 
 The cryptic words that guided pilots on the Hiroshima bombing mission http://t.co/nSS5L64cvR #canada
---



In [15]:
#Splitting data into Train and Validation
from sklearn.model_selection import train_test_split

train_sentances,val_sentances,train_labels,val_labels=train_test_split(
                                                      train_df_shuffled['text'].to_numpy(),
                                                      train_df_shuffled['target'].to_numpy(),
                                                      test_size=0.1,
                                                      random_state=42)

In [16]:
# Check lengths
len(train_sentances),len(val_sentances),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# **Convert Text into Numbers**

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(
                                  max_tokens=10000, # Set max_tokens to a value
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=True,
                                   )

In [18]:
max_vocab_length=100   # Max number of words to have in vocabulary
max_length=15  # Max length our sequence will be
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=max_vocab_length,
                                  pad_to_max_tokens=True)

In [19]:
# Fit the Text Vectorizer
text_vectorizer.adapt(train_sentances)

In [20]:
# Create a sample sentance and tokenize it
sample_sentance=random.choice(train_sentances)
print(f' The Original Sentance \n {sample_sentance} \n The Text Vectorized Version \n')
text_vectorizer([sample_sentance])

 The Original Sentance 
 The insides of the explosion box..
Get it customized the way you want!

#dcubecrafts #greetingcardsÛ_ https://t.co/T07qxP5cBE 
 The Text Vectorized Version 



<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[ 2,  1,  6,  2,  1,  1, 52, 15,  1,  2,  1, 12,  1,  1,  1,  1,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]])>

In [21]:
# Get uniqe words in vocab
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'The Number of words in vocab are : {len(words_in_vocab)}')
print(f'The Five most common words in vocab are : {top_5_words}')
print(f'The Five least words in vocab are : {bottom_5_words}')

The Number of words in vocab are : 100
The Five most common words in vocab are : ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
The Five least words in vocab are : [np.str_('buildings'), np.str_('rt'), np.str_('first'), np.str_('cant'), np.str_('see')]


In [22]:
# Creating a embedding layer that turns text vectorized into the embeding vectors
from tensorflow.keras.layers import Embedding

embedding=Embedding(input_dim=max_vocab_length,
                    output_dim=128,
                    )

In [23]:
# Get Random Sentance from trainning set
random_sentance=random.choice(train_sentances)
print(f'Original Text \n {random_sentance}\n')
sample_embedding=embedding(text_vectorizer([random_sentance]))
sample_embedding

Original Text 
 I'm loving this classic barn shot! We may or may not have got electrocuted got stung a few times and stepped in... http://t.co/X6aSGRjsWC



<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
array([[[ 0.03014734, -0.04384613, -0.02922777, ...,  0.02506921,
         -0.02240675, -0.04163507],
        [ 0.02266918, -0.01995186,  0.00289994, ..., -0.0473236 ,
          0.03376264, -0.0265023 ],
        [ 0.0411945 ,  0.02148844,  0.03441937, ..., -0.0049775 ,
         -0.04240881,  0.00167096],
        ...,
        [-0.03490497,  0.02895408, -0.01048367, ..., -0.00315329,
         -0.03401694, -0.04065144],
        [-0.03490497,  0.02895408, -0.01048367, ..., -0.00315329,
         -0.03401694, -0.04065144],
        [-0.03490497,  0.02895408, -0.01048367, ..., -0.00315329,
         -0.03401694, -0.04065144]]], dtype=float32)>

# ***Lets Now Create our Baseline Model***
its common to use ML model first or non DL model and then switch to DL model

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
    ('tfid',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

model_0.fit(train_sentances,train_labels)
baseline_score=model_0.score(val_sentances,val_labels)

print(f'Our Baseline Model Acheives Accuracy of : {baseline_score*100:.2f}')

Our Baseline Model Acheives Accuracy of : 79.27


In [25]:
# Model Predictions
baseline_preds=model_0.predict(val_sentances)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# **Creating an evaluation function**

In [26]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true=y_true,y_pred=y_pred)
  model_precission,model_recall,model_f1,_=precision_recall_fscore_support(y_true=y_true,y_pred=y_pred,average='weighted')

  model_results={
     "accuracy":model_accuracy,
     "precsion":model_precission,
     "recall":model_recall,
     'f1_score':model_f1
  }
  return model_results

In [27]:
# Get Baseline Results
baseline_results=calculate_results(y_true=val_labels,y_pred=baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# **Model1 : Feed Forward Neural Network ( Dense Model)**

In [28]:
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers

In [29]:
SAVE_DIR="ModelLogs"

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # Add this layer to reduce dimensions
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_Dense")
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,929 (50.50 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_1_history=model_1.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_1_dense')])

Saving TensorBoard log files to: ModelLogs/model_1_dense/20251026-071603
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5658 - loss: 0.6826 - val_accuracy: 0.5433 - val_loss: 0.6800
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5769 - loss: 0.6699 - val_accuracy: 0.5787 - val_loss: 0.6627
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6092 - loss: 0.6551 - val_accuracy: 0.6483 - val_loss: 0.6388
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6339 - loss: 0.6382 - val_accuracy: 0.6627 - val_loss: 0.6221
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6676 - loss: 0.6132 - val_accuracy: 0.6339 - val_loss: 0.6342


In [31]:
model_1.evaluate(val_sentances,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6207 - loss: 0.6584


[0.6341687440872192, 0.6338582634925842]

In [32]:
model_1_predict_probs=model_1.predict(val_sentances)
model_1_predict_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[0.41432133],
       [0.5407893 ],
       [0.6024225 ],
       [0.23142602],
       [0.32359833],
       [0.50702536],
       [0.590639  ],
       [0.5356057 ],
       [0.5429497 ],
       [0.18834917],
       [0.36257634],
       [0.406003  ],
       [0.45248067],
       [0.34041524],
       [0.4996337 ],
       [0.3665099 ],
       [0.18877049],
       [0.2715337 ],
       [0.3007244 ],
       [0.38773206],
       [0.17472044],
       [0.09013733],
       [0.2831259 ],
       [0.33199504],
       [0.3771925 ],
       [0.49195072],
       [0.0909083 ],
       [0.4080649 ],
       [0.16759352],
       [0.20154598],
       [0.32867745],
       [0.33560598],
       [0.4103986 ],
       [0.34668675],
       [0.4022485 ],
       [0.10235772],
       [0.50944847],
       [0.1847616 ],
       [0.17366764],
       [0.5548491 ],
       [0.2538151 ],
       [0.47654364],
       [0.29197547],
       [0.39330623],
       [0.25886902],
       [0.45664784],
       [0.32622457],
       [0.626

In [33]:
model_1_predict_probs.shape

(762, 1)

In [34]:
model_1_predict_probs[0]

array([0.41432133], dtype=float32)

In [35]:
# Make the preictions comaparable with the actaul labels
model_1_predict_probs=tf.squeeze(tf.round(model_1_predict_probs))
model_1_predict_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [36]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

# ***Calculate Model Results***

In [37]:
model_1_results=calculate_results(y_true=val_labels,y_pred=model_1_predict_probs)
model_1_results

{'accuracy': 0.6338582677165354,
 'precsion': 0.6800743922219686,
 'recall': 0.6338582677165354,
 'f1_score': 0.5880996237312797}

In [38]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

# ***Lets Visualize the embedding that Model learned using Tensorflow Projector***

In [39]:
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(100,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [40]:
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,789 (151.52 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,860 (101.02 KB)

In [41]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
embed_weights,print(embed_weights.shape)

(100, 128)


(array([[-0.0474328 , -0.01829541,  0.02090177, ..., -0.03636983,
          0.0151578 , -0.06661812],
        [ 0.21566932,  0.143458  , -0.1559395 , ...,  0.1385901 ,
         -0.10536648,  0.16851062],
        [-0.15702176, -0.18805403,  0.21020553, ..., -0.17485799,
          0.21768025, -0.18193826],
        ...,
        [ 0.05051449,  0.05950725, -0.06682415, ...,  0.02703283,
         -0.0593372 , -0.00679595],
        [-0.33486792, -0.4179492 ,  0.39716566, ..., -0.3620644 ,
          0.35633844, -0.27178305],
        [-0.24780518, -0.27161008,  0.33804557, ..., -0.29235393,
          0.32291746, -0.28521264]], dtype=float32),
 None)

Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).  This code is from the tensorflow documentation Here[https://www.tensorflow.org/text/guide/word_embeddings]

In [42]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [43]:
# After this download step use projector tensorflow to visualzize it
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""


"\ntry:\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception:\n  pass\n"

# ***MODEL2 =  LSTM [LONG SHORT TERM MEMORY]***

In [44]:
from tensorflow.keras import layers


In [45]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(inputs,outputs)

In [46]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [47]:
model_2.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,457 (388.50 KB)

 Trainable params: 99,457 (388.50 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model_2_history=model_2.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: ModelLogs/model_2_LSTM/20251026-071613
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5694 - loss: 0.6849 - val_accuracy: 0.5433 - val_loss: 0.6894
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5765 - loss: 0.6822 - val_accuracy: 0.5433 - val_loss: 0.6899
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5749 - loss: 0.6826 - val_accuracy: 0.5433 - val_loss: 0.6906
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5759 - loss: 0.6820 - val_accuracy: 0.5433 - val_loss: 0.6913
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5829 - loss: 0.6797 - val_accuracy: 0.5433 - val_loss: 0.6903


In [49]:
model_2_pred_probs=model_2.predict(val_sentances)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


array([[0.43529],
       [0.43529],
       [0.43529],
       [0.43529],
       [0.43529],
       [0.43529],
       [0.43529],
       [0.43529],
       [0.43529],
       [0.43529]], dtype=float32)

In [50]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [51]:
model_2_results=calculate_results(y_true=val_labels,y_pred=model_2_preds)
model_2_results

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.5433070866141733,
 'precsion': 0.29518259036518074,
 'recall': 0.5433070866141733,
 'f1_score': 0.38253254057528524}

In [52]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# ***As it is clear our baseline is more accurate***

# ***Model_3 GRU (Gated Recurrent Unit)***

In [53]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GRU(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_3=tf.keras.Model(inputs,outputs)


In [54]:
model_3.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,273 (212.00 KB)

 Trainable params: 54,273 (212.00 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [56]:
model_3_history=model_3.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_GRU")])

Saving TensorBoard log files to: ModelLogs/model_GRU/20251026-071634
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5561 - loss: 0.6852 - val_accuracy: 0.5433 - val_loss: 0.6906
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5649 - loss: 0.6856 - val_accuracy: 0.5433 - val_loss: 0.6902
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5754 - loss: 0.6831 - val_accuracy: 0.5433 - val_loss: 0.6963
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5764 - loss: 0.6821 - val_accuracy: 0.5433 - val_loss: 0.6920
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5871 - loss: 0.6780 - val_accuracy: 0.5433 - val_loss: 0.6896


In [57]:
model_3_pred_probs=model_3.predict(val_sentances)
model_3_pred_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


array([[0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.44613856],
       [0.446

In [58]:
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [59]:
model_3_results=calculate_results(val_labels,model_3_pred)
model_3_results

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.5433070866141733,
 'precsion': 0.29518259036518074,
 'recall': 0.5433070866141733,
 'f1_score': 0.38253254057528524}

# ***GRU Does not beat the baseline***
lets try something else

# ***Model_4 Bidirectional Model***

In [60]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_4=tf.keras.Model(inputs,outputs)

model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_4.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,745 (436.50 KB)

 Trainable params: 111,745 (436.50 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model_4_history=model_4.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bi_directional")])

Saving TensorBoard log files to: ModelLogs/model_4_bi_directional/20251026-071651
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6953 - loss: 0.5949 - val_accuracy: 0.6811 - val_loss: 0.6161
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7149 - loss: 0.5722 - val_accuracy: 0.6877 - val_loss: 0.5936
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7105 - loss: 0.5625 - val_accuracy: 0.6929 - val_loss: 0.5966
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7260 - loss: 0.5486 - val_accuracy: 0.6903 - val_loss: 0.5888
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7294 - loss: 0.5459 - val_accuracy: 0.7113 - val_loss: 0.5773


In [62]:
model_4_pred_probs=model_4.predict(val_sentances)
model_4_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


array([[0.47912097],
       [0.6801736 ],
       [0.5746057 ],
       [0.27526224],
       [0.3199024 ],
       [0.6509503 ],
       [0.7819601 ],
       [0.74651855],
       [0.7441455 ],
       [0.15885091]], dtype=float32)

In [63]:
model_4_pred=tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [64]:
model_4_results=calculate_results(val_labels,model_4_pred)

In [65]:
model_4_results

{'accuracy': 0.7112860892388452,
 'precsion': 0.7276513311777119,
 'recall': 0.7112860892388452,
 'f1_score': 0.6991424210612955}

# ***Still Not enough to beat the baseline***

**1D Convolutional Neural Network**

In [66]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,strides=1,activation='relu',padding='valid')(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs,name='Model_5_Conv1D')

In [67]:
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [68]:
model_5_history=model_5.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6956 - loss: 0.5900 - val_accuracy: 0.6850 - val_loss: 0.6013
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7177 - loss: 0.5574 - val_accuracy: 0.6759 - val_loss: 0.5932
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7309 - loss: 0.5478 - val_accuracy: 0.6811 - val_loss: 0.5929
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7248 - loss: 0.5481 - val_accuracy: 0.6929 - val_loss: 0.5851
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7398 - loss: 0.5289 - val_accuracy: 0.6955 - val_loss: 0.5870


In [69]:
model_5_pred_probs=model_5.predict(val_sentances)
model_5_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


array([[0.6800068 ],
       [0.46601665],
       [0.52359325],
       [0.38178402],
       [0.46703717],
       [0.66686726],
       [0.96955734],
       [0.80672663],
       [0.6957864 ],
       [0.13353898]], dtype=float32)

In [70]:
model_5_pred=tf.squeeze(tf.round(model_5_pred_probs))
model_5_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [71]:
model_5_results=calculate_results(val_labels,model_5_pred)
model_5_results

{'accuracy': 0.6955380577427821,
 'precsion': 0.694828392311652,
 'recall': 0.6955380577427821,
 'f1_score': 0.6934264844332912}

# **Still does not beat the Baseline**
 Lets try a pretrained  model from tensorflow hub
 Using *USE: Unviversal Sentence Encoder*

In [72]:
import tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [ ]:
url='https://tfhub.dev/google/universal-sentence-encoder/4'

In [73]:
embed_samples=embed([sample_sentance])
print(embed_samples[0][:50])

tf.Tensor(
[-0.03066502  0.0496372  -0.05217693  0.00139907 -0.00926552  0.00673183
  0.01304122  0.06996512  0.06852724  0.07824161 -0.00789557 -0.0040019
  0.00460476 -0.01420973  0.03097453  0.0219287   0.00318545  0.02783472
 -0.08563413 -0.03833722 -0.04447577  0.04989638  0.03295439  0.01984563
 -0.03200996 -0.03538054 -0.06820709  0.03808664  0.03749126  0.01803972
 -0.02028149 -0.01247546 -0.01853     0.0129871   0.01497612  0.06763671
 -0.092466    0.00197926 -0.01782483 -0.03774925  0.00956515 -0.0329027
 -0.06632379 -0.01825559 -0.04180375  0.00383663  0.01642472  0.02767501
 -0.066098   -0.00594822], shape=(50,), dtype=float32)


In [ ]:
sentence_encoder_layer=hub.KerasLayer()